# How to start

Before starting you must:
- Have conda installed
- `conda env create -f ess-notebooks-latest.yml python=3.7` . The yaml environment file is part of this repository.
- fetch the data `git clone git@github.com:scipp/ess-notebooks-data.git` somewhere local 
- Generate the `dataconfig.py` file using `make_config.py` located in same directory as this notebook. In general, you simply need to point `make_config.py` to the root directory of data you cloned above. Refer to the help `make_config.py --help` for more information. 

Converted to use scipp and notebook from [this repository](https://github.com/scipp/ess-legacy).

For Table of Contents install Jupyter extensions then reload the notebook:
`conda install -c conda-forge jupyter_contrib_nbextensions`
`jupyter contrib nbextension install --user`
`jupyter nbextension enable toc2/main`

# Experimental Summary

This script has been developed to measure local strain ε defined as ε = ΔL/L0 in a FCC steel sample under elastic strain in a stress rig. Measured at V20, HZB, Berlin, September 2018 by Peter Kadletz.

λ = 2dsinθ, where 2θ = π (transmission), edges characterise the Bragg condition and hence λ = 2d. Therefore strain is easily computed from the wavelength measurement of of a Bragg edge directly, using un-loaded vs loaded experimental runs (and reference mesurements). The known Miller indices of the crystal structure (FCC) are used to predict the wavelength where the Bragg edges should exist, which is bound by the reachable wavelength extents for the instrument. This provides an approximate region to apply a fit.  A complement error function is used to fit each Bragg edge, and a refined centre location (λ) for the edge is used in the strain measurement. Because each bragg edge can be identified individually, one can determine anisotropic strain across the unit cell in the reachable crystallographic directions. In addition the image processing allows for spacial grouping so localised effects, such as those on unconstrained edges of the sample or in necking regions of the sample can be treated seperately. The plotted outputs in the script aim to capture this.


# Script setup

In [ ]:
# try:
#     import scipp
# except ImportError as e:
#     print("scipp is not available in the PYTHONPATH")
#     raise e

# try:
#     import mantid
# except ImportError as e:
#     print("mantid is not available in the PYTHONPATH")
#     raise e

# try:
#     import dataconfig
# except ImportError as e:
#     print(
#         "dataconfig is not available. Make sure you have generated it with `make_config.py`."
#     )
#     raise e

## Floating point precision 

In [ ]:
import numpy as np
float_type = np.float32


# Helper to determine scipp dtype from np dtype
def scipp_dtype(dtype):
    return sc.Variable(value=0, dtype=dtype).dtype

## Set input and output dirs

If your input directory has a different structure this is the cell to modify. 
Additionally the output directory can be renamed too.

In [ ]:
# Lets get everything set up
import os
import sys

import scipp as sc
import numpy as np
from scipp.plot import plot
from dress import wfm
import matplotlib.pyplot as plt

import ess.v20.imaging as imaging
import ess.v20.imaging.operations as operations
import dataconfig
from scipy import ndimage, signal

local_data_path = os.path.join('ess', 'v20', 'imaging', 'gp2-stress-experiments')
data_dir = os.path.join(dataconfig.data_root, local_data_path)
output_dir = os.path.join(dataconfig.data_root, 'output')
instrument_file = os.path.join(data_dir, 'V20_Definition_GP2.xml')

tofs_path = os.path.join(data_dir, 'GP2_Stress_time_values.txt')
raw_data_dir = os.path.join(data_dir)

if not os.path.exists(data_dir):
    raise FileNotFoundError("The following data directory does not exist,"
                            f" check your make_config.py:\n{data_dir}")

## Geometry

In [ ]:
geometry = sc.Dataset()
sc.compat.mantid.load_component_info(geometry, instrument_file)

## Reduction Options

In [ ]:
# # Customisable Options:

# # defining grouping of 2D detector pixels
# nx_source = 324
# ny_source = 324
# grouping_number = 27
# nx_target = grouping_number
# ny_target = grouping_number

# bin_width = (64 * 2.5)  # μS

# # Rebin regions for each of the 5 WFM frames. Values are in detector time : μS
# # in the format of [bin-start, bin-end, bin width].
# # used to crop each image, before stitching them together
# frame_parameters = [{
#     "start": 15450,
#     "stop": 22942,
#     "step": bin_width
# }, {
#     "start": 24800,
#     "stop": 32052,
#     "step": bin_width
# }, {
#     "start": 33791,
#     "stop": 40084,
#     "step": bin_width
# }, {
#     "start": 41763,
#     "stop": 47457,
#     "step": bin_width
# }, {
#     "start": 49315,
#     "stop": 54500,
#     "step": bin_width
# }, {
#     "start": 56500,
#     "stop": 58360,
#     "step": bin_width
# }]

# # Used to rebin the summed frame in order to
# # cut off frames that contain no data
# rebin_parameters = {"start": 8550, "stop": 26000, "step": bin_width}

# # Used to shift the cropped frames so that their bins overlap
# # before summing them together into a single frame
# frame_shift_increments = [-6630, -2420, -2253, -2095, -1946, -1810]
# frame_shift_increments = [float(i)
#                           for i in frame_shift_increments]  # Work around #1114

# Pulse references
pulse_number_reference = 1.0 / 770956
pulse_number_sample = 1.0 / 1280381
pulse_number_sample_elastic = 1.0 / 2416839
pulse_number_sample_plastic = 1.0 / 2614343

# units of transmission, all pixels with transmission higher masking threshold are masked
masking_threshold = 0.80

# Toggles outputting masked and sliced tiff stacks
output_tiff_stack = False

# Experiment Metadata
measurement_number = 11

# Reduction

## Load the data files and instrument geometry

In [ ]:
# Load time bins from 1D text file
ds = sc.Dataset()
ds.coords["t"] = sc.Variable(["t"],
                             unit=sc.units.us,
                             values=imaging.read_x_values(tofs_path, skiprows=1, usecols=1, delimiter='\t'),)
ds.coords["t"] *= 1e3

# Load tiff stack
def load_and_scale(folder_name, scale_factor):
    to_load = os.path.join(raw_data_dir, folder_name)
    variable = imaging.tiffs_to_variable(to_load, dtype=float_type)
    variable *= scale_factor
    return variable

ds["reference"] = load_and_scale(folder_name="R825-open-beam",
                                 scale_factor=pulse_number_reference)
ds["sample"] = load_and_scale(folder_name="R825",
                              scale_factor=pulse_number_sample)
ds["sample_elastic"] = load_and_scale(folder_name="R825-600-Mpa",
                                      scale_factor=pulse_number_sample_elastic)

In [ ]:
geometry = sc.Dataset()
sc.compat.mantid.load_component_info(geometry, instrument_file)
geom = sc.Dataset(coords={"sample-position": geometry.coords["sample-position"],
                          "source-position": geometry.coords["source-position"]})
geom.coords["position"] = sc.reshape(geometry.coords['position'], dims=['y', 'x'],
                                     shape=tuple(ds["sample"]["t", 0].shape))
geom.coords["x"] = sc.geometry.x(geom.coords["position"])["y", 0]
geom.coords["y"] = sc.geometry.y(geom.coords["position"])["x", 0]
ds = sc.merge(ds, geom)
ds

## Raw data visualization

In [ ]:
plot(ds["sample"])

## Converting time coordinate to TOF

Use the instrument geometry and chopper cascade parameters to compute time-distance diagram.

In [ ]:
plt.ion()
v20setup = wfm.v20.setup()
v20setup['info']['detector_positions'] = {
    "GP2": -ds.coords['source-position'].value[2] + v20setup['info']['wfm_choppers_midpoint'] + sc.geometry.z(
    geom.coords["position"])['x', 0]['y', 0].value}

frames = wfm.get_frames(instrument=v20setup, plot=True)
frames

In [ ]:
plt.ioff()
fig1, ax1 = plt.subplots()
plot(sc.sum(sc.sum(ds["reference"], 'x'), 'y'), ax=ax1)
for i in range(len(frames["GP2"]["left_edges"])):
    ax1.axvspan(frames["GP2"]["left_edges"][i], frames["GP2"]["right_edges"][i], color="C{}".format(i), alpha=0.3)
fig1.canvas.draw_idle()
fig1.canvas

Extract the sections in the original data using value-based slicing and shift the coordinates:

In [ ]:
sections = {}
for key in ds:
    sections[key] = []
    for i in range(len(frames["GP2"]["left_edges"])):
        section = ds[key]["t",
                             frames["GP2"]["left_edges"][i]*sc.units.us:frames["GP2"]["right_edges"][i]*sc.units.us].copy()
        section.coords["t"] += frames["GP2"]["shifts"][i]*sc.units.us
        section.rename_dims({'t': 'tof'})
        sections[key].append(section)

In [ ]:
plot({"frame{}".format(i): sc.sum(sc.sum(sections["reference"][i], 'x'), 'y')
      for i in range(len(sections["reference"]))})

To stitch the data, we make a common container with a TOF axis spanning the entire range, and sum the counts from the different frames.

In [ ]:
ntof = 512
stitched = sc.Dataset()
stitched.coords["tof"] = sc.Variable(["tof"],
                             unit=sc.units.us,
                             values=np.linspace(9.0e3, 5.0e4, ntof + 1))
for key in ds.coords:
    if key != "t":
        stitched.coords[key] = ds.coords[key]
# Make empty data container
for key in ds:
    stitched[key] = sc.zeros(dims=["tof", "y", "x"], shape=[ntof] + ds.coords["position"].shape,
                                 variances=True, unit=sc.units.counts)
# Sum counts from different frames
for key in sections:
    for sec in sections[key]:
        stitched[key] += sc.rebin(sec, 'tof', stitched.coords["tof"])
stitched

In [ ]:
plot(sc.sum(sc.sum(stitched, 'x'), 'y'))

## Normalization

In [ ]:
# Normalize by open beam
normalized = stitched / stitched["reference"]
del normalized["reference"]

replacement = sc.Variable(value=0.0, variance=0.0)
kwargs = {"nan": replacement, "posinf": replacement, "neginf": replacement}
for k in normalized.keys():
    sc.nan_to_num(normalized[k].data, out=normalized[k].data, **kwargs)

plot(sc.sum(sc.sum(normalized, 'x'), 'y'))

## Convert to wavelength

In [ ]:
wavelength = sc.neutron.convert(normalized, "tof", "wavelength")

In [ ]:
plot(wavelength["sample"], axes={'x': "wavelength"})